In [1]:
import scCube
from scCube import scCube
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

/home/qjy/anaconda3/envs/sccube/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = scCube()

In [3]:
# slice: 151507
sc_data = pd.read_csv('data/DLPFC/processed/' + '151507' + '_data.csv', index_col=0)
sc_meta = pd.read_csv('data/DLPFC/processed/' + '151507' + '_meta.csv', index_col=0)
    
sc_adata = model.pre_process(
    sc_data=sc_data, 
    sc_meta=sc_meta,
    is_normalized=False)

# slice: 151676
sc_data = pd.read_csv('data/DLPFC/processed/' + '151676' + '_data.csv', index_col=0)
sc_meta = pd.read_csv('data/DLPFC/processed/' + '151676' + '_meta.csv', index_col=0)
    
sc_adata2 = model.pre_process(
    sc_data=sc_data, 
    sc_meta=sc_meta,
    is_normalized=False)

the input is count matrix, normalizing it firstly...
the input is count matrix, normalizing it firstly...


In [4]:
dict(sc_adata2.obs['Cell_type'].value_counts())

{'Layer_3': 836,
 'Layer_5': 649,
 'Layer_6': 616,
 'WM': 533,
 'Layer_1': 289,
 'Layer_2': 254,
 'Layer_4': 254}

In [5]:
# load model trained from slice:151507
generate_sc_meta, generate_sc_data = model.load_vae_and_generate_cell(
        sc_adata=sc_adata,
        celltype_key='Cell_type',
        cell_key='Cell',
        target_num=dict(sc_adata2.obs['Cell_type'].value_counts()),
        hidden_size=128,
        load_path='/home/qjy/workspace/scCube/save_model/slice151507_epoch100000.pth',
        used_device='cuda:0')

generating by the targeted proportion of cell types...
loading model from /home/qjy/workspace/scCube/save_model/slice151507_epoch100000.pth
vae loading done!


Generate Epoch: 1: 100%|██████████| 3431/3431 [00:00<00:00, 4083.78it/s]


generated done!
data have been prepared!


In [6]:
generate_sc_data, generate_sc_meta = model.generate_pattern_reference(
        sc_adata=sc_adata2,
        generate_sc_data=generate_sc_data,
        generate_sc_meta=generate_sc_meta,
        celltype_key='Cell_type',
        spatial_key=['x', 'y'],
        cost_metric='sqeuclidean'
    )

In [7]:
# saving
generate_sc_data.to_csv('result/sccube_DLPFC_151676_epoch100000_data_trained_on_other.csv')
generate_sc_meta.to_csv('result/sccube_DLPFC_151676_epoch100000_meta_trained_on_other.csv')